# Data Cleaning

Info: Data source: [https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset](https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset)

In [5]:
import pandas as pd
import numpy as np
import operator
from sklearn.impute import SimpleImputer
import os
from pathlib import Path

In [26]:
original_wd = os.getcwd()
temp_wd = os.getcwd() + "/../data/raw/"

os.chdir(temp_wd)

my_file = Path(temp_wd + "bank-additional-full.csv")
if my_file.exists():
    print("file \"bank-additional-full.csv\" already exists! updating file...")
    !rm bank-additional-full.csv
!unzip bank-marketing-campaigns-dataset.zip

os.chdir(original_wd)

file "bank-additional-full.csv" already exists! updating file...
Archive:  bank-marketing-campaigns-dataset.zip
  inflating: bank-additional-full.csv  


In [7]:
df = pd.read_csv("../data/raw/bank-additional-full.csv", sep=';')

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [10]:
df.shape

(41188, 21)

**Check for duplicates**

In [11]:
duplicates = tuple(map(operator.sub, df.shape, df.drop_duplicates().shape)) # or alternatively:
duplicates = tuple(map(lambda i, j: i - j, df.shape, df.drop_duplicates().shape)) 
duplicates

(12, 0)

In [12]:
print('There are ' + str(duplicates[0]) + ' duplicate rows in the dataset.')

There are 12 duplicate rows in the dataset.


**Drop duplicates in the dataset**

In [13]:
df = df.drop_duplicates()

In [14]:
df.shape

(41176, 21)

**Check for missing values**

In [15]:
df.isna().sum() # or equivalently: df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

**Drop missing values**

In [16]:
df = df.dropna() # drop incomplete rows (actually not needed, since there are no missing values)

In [17]:
df.shape

(41176, 21)

**Inspect other unknown values**

Info: There are several missing values in some categorical attributes, all coded with the "unknown" label.

In [18]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [19]:
columns_with_unknown_values = []

for col in df.columns:
    num_entries = len(df[df[col] == 'unknown'])
    if num_entries > 0:
        columns_with_unknown_values += [col]
        print('Column "' + col + '" has ' + str(num_entries) + ' unknown entries.')
        
if len(columns_with_unknown_values) == 0:
    print('no columns with unknown values found')

Column "job" has 330 unknown entries.
Column "marital" has 80 unknown entries.
Column "education" has 1730 unknown entries.
Column "default" has 8596 unknown entries.
Column "housing" has 990 unknown entries.
Column "loan" has 990 unknown entries.


/home/stefan/progs/installations/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [20]:
columns_with_unknown_values

['job', 'marital', 'education', 'default', 'housing', 'loan']

**If preferred, impute unknown values with the most frequent values**

In [21]:
impute_unknown_values = False

if impute_unknown_values:
    if len(columns_with_unknown_values) > 0:
        impute_missing = SimpleImputer(missing_values='unknown', strategy='most_frequent') # imputes missing data; possible strategies: mean, median, most_frequent, ...
        df[columns_with_unknown_values] = impute_missing.fit_transform(df[columns_with_unknown_values])

**If preferred, delete all rows where at least one column has the value 'unknown'**

In [22]:
drop_rows_unknown_values = False

if drop_rows_unknown_values:
    df = df[(df.iloc[:, :] != 'unknown').all(axis=1)] # iloc[<row selection>, <column selection>]
df.shape

(41176, 21)

**If preferred, just consider missing attribute values with value 'unknown' as a class label**

In [23]:
if not impute_unknown_values and not drop_rows_unknown_values:
    print('Info: Attribute values with the value "unknown" will be considered as an additional class label.')

Info: Attribute values with the value "unknown" will be considered as an additional class label.


**Export cleaned csv file**

In [24]:
df.to_csv("../data/clean/clean.csv", index=False)